# Tasas de Interés

## Librerías

In [2]:
from finrisk import QC_Financial_3 as Qcf
from IPython.display import Image
from IPython.core.display import HTML

## Convenciones de Tasas

Consideremos la siguiente situación:

- El día 10-09-2020, nos conectamos al sitio web de un banco de la plaza para contratar un depósito a plazo en CLP, a 30 días, por 1 MM CLP.
- La página del banco nos ofrece una tasa de 0,11% para ese plazo.
- ¿Tenemos claro cuál es el negocio que nos están ofreciendo?

In [3]:
Image(url="img/20200910_dap_online_0.png", width=500, height=400)

Al apretar el botón para **Continuar** se despliega el siguiente resultado:

![DAP Online](img/20200910_dap_online.png)

Veamos como se llega a este resultado. Nuestra solicitud fue:

In [4]:
monto_inicial = 1000000
plazo = 30

La pantalla inicial indica que la tasa para este plazo es:

In [5]:
valor_tasa = .0011

¿Porqué el plazo de la operación simulada es de 33 días?

In [6]:
fecha_inicial = Qcf.QCDate(10, 9, 2020)

fecha_final = fecha_inicial.add_days(plazo)

print(fecha_final.description(True))

print(fecha_final.week_day())

10-10-2020
SAT


Sumando 30 días al 10-09-2020 llegamos al 10-10-2020 que es un sábado (`SAT`). Para corregir esto, consideremos dentro del cálculo los días feriados.

In [7]:
calendario = Qcf.BusinessCalendar(fecha_inicial, 1)

fecha_final = calendario.next_busy_day(fecha_final)

print(fecha_final.description(True))

print(fecha_final.week_day())

12-10-2020
MON


Vemos que, al pasar a la fecha hábil siguiente, llegamos al 12-10-2020. Sin embargo, la fecha final del depósito simulado es el 13-10-2020. El objeto calendario considera por default como feriados los días sábado y domingo, pero el lunes 12-10-2020 también es feriado y hay que agregarlo manualmente al calendario.

In [8]:
calendario.add_holiday(fecha_final) # Se agrega el 12-10-2020 como feriado al calendario.

fecha_final = calendario.next_busy_day(fecha_final) # Se recalcula la fecha final.

print(f'Fecha final: {fecha_final.description(True)}.')

print(f'Día final: {fecha_final.week_day()}.')

plazo_op = fecha_inicial.day_diff(fecha_final)

print(f'Plazo: {plazo_op} días.')

Fecha final: 13-10-2020.
Día final: TUE.
Plazo: 33 días.


¿Cómo se calcula el interés? La convención de tasa en Chile para depósitos a plazo en CLP es Lineal Act/30.

In [9]:
yf = Qcf.QCAct30() # Act/30

wf = Qcf.QCLinearWf() # Lineal

tasa = Qcf.QCInterestRate(valor_tasa, yf, wf) # Tasa con ese valor y convención

Así, el monto final se calcula como:

In [10]:
monto_final = monto_inicial * tasa.wf(fecha_inicial, fecha_final) # wf: factor de capitalización

print(f'Monto final: {monto_final:,.0f}')

Monto final: 1,001,210


Veamos explícitamente este cálculo:

In [11]:
print(f'Monto final: {monto_inicial * (1 + valor_tasa * plazo_op / 30): ,.0f}')

Monto final:  1,001,210


El monto final, entonces, se calcula utilizando la siguiente fórmula:

$$
\begin{equation}
M_{final}=M_{inicial}\cdot\left(1+r\cdot\frac{fecha_{final}-fecha_{inicial}}{30}\right)
\end{equation}
$$

donde:

- $M_{inicial}$ es el monto inicial del depósito.
- $M_{final}$ es el monto final del depósito.
- $r$ es el valor de la tasa ofrecida por el banco.
- $fecha_{final}-fecha_{inicial}$ representa la diferencia real en días entre las fechas inicial y final del depósito.

¿Porqué se usa la fórmula anterior?

Por convención:

- en Chile las tasas de los depósitos a plazo en CLP se cotizan de esta forma.
- La fórmula tiene distintas componentes, las cuales, en su conjunto, definen el tipo de tasa de interés asociado a los depósitos en CLP.

Abstrayendo la estructura de la fórmula (1) podemos escribir:

$$
\begin{equation}
M_{final}=M_{inicial}\cdot wf(valorTasa, yf(fecha_{inicial},fecha_{final}))
\end{equation}
$$

Donde:

- $yf$ es la función *fracción de año* que determina el período de tiempo que transcurre entre ambas fechas y
- $wf$ es la función *factor de capitalización* que establece el factor a aplicar al monto inicial de la inversión para obtener el monto final de la inversión. 

Para el cálculo de la fracción de año, existen distintas maneras de contar los días:
    
- Actual (días reales o la diferencia de fechas que acabamos de ver)
- En base 30 (todos los meses tienen 30 días)

y distintas maneras de representar un *período* de tasa:

- Base 30 (meses)
- Año de 360 días
- Año de 365 días

También hay distintas maneras de calcular el factor de capitalización, entre ellas:

- Lineal:

$$
\begin{equation}
1+r_{lineal}\cdot yf(fecha_{inicial},fecha_{final})
\end{equation}
$$

- Compuesto

$$
\begin{equation}
\left(1+r_{compuesto}\right)^{yf(fecha_{inicial},fecha_{final})}
\end{equation}
$$

- Exponencial
$$
\begin{equation}
\exp\left(r_{exponencial}\cdot yf\left(fecha_{inicial},fecha_{final}\right)\right)
\end{equation}
$$

Plazo de la Tasa:

- En Chile se suele hablar de la tasa a 30 días, 90 días, 180 días etc. Como vimos en el ejemplo anterior, no todos los días podemos tener una tasa a 30 días (por el vencimiento en día inhábil).
- Esto hace difícil la comparación en el tiempo de las tasas a un mismo plazo (de hecho no todos los días podemos tener tasa a un cierto plazo).
- En los mercados desarrollados, esto se resuelve a través del concepto de plazo estándar o Tenor.

Tenor

- Por ejemplo, en vez de tasa a 30 días se habla de la tasa a 1 mes (1M) que significa, por ejemplo (es una convención de mercado), que la tasa es al mismo día de hoy, pero del mes siguiente.
- Si ese día es inhábil se adopta una convención que puede ser ir al próximo día hábil. Así tenemos también 3M, 6M, 1Y (un año) etc.

In [11]:
un_mes = Qcf.Tenor('1M')
print(f'Un mes sin corrección: {fecha_inicial.add_months(un_mes.get_months()).description(True)}')

temp = calendario.next_busy_day(fecha_inicial.add_months(un_mes.get_months()))
print(f'Un mes con corrección al día hábil siguiente: {temp.description(True)}')

Un mes sin corrección: 10-10-2020
Un mes con corrección al día hábil siguiente: 13-10-2020


Consideremos la siguiente inversión.

- Fecha inicial: 10-09-2020
- Plazo de la inversión: 1Y (corrección al día hábil siguiente).
- En la fecha final recibo 10 CLP por cada 100 CLP invertidos.
- Dado el plazo de la inversión, ésta queda definida, por su factor de capitalización $\frac{110}{100}=1.1$.

## Ejercicio

Utilizando la librería `QC_Financial_3` replicar los valores de la siguiente tabla:

In [12]:
Image(url="img/tabla_ejercicio.png", width=550, height=440)